In [18]:
import argparse
import qlib
import ruamel.yaml as yaml
from qlib.utils import init_instance_by_config
import pandas as pd
import torch
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)

In [2]:
parser = argparse.ArgumentParser(allow_abbrev=False)
parser.add_argument("--seed", type=int, default=1000, help="random seed")
parser.add_argument("--config_file", type=str, default="us158_nassp_epoch100.yaml", help="config file")
parser.add_argument("-f")
args = parser.parse_args()


seed = args.seed
config_file = args.config_file

# set random seed
with open(config_file) as f:
    config = yaml.safe_load(f)

# seed_suffix = "/seed1000" if "init" in config_file else f"/seed{seed}"
seed_suffix = ""
config["task"]["model"]["kwargs"].update(
    {"seed": seed, "logdir": config["task"]["model"]["kwargs"]["logdir"] + seed_suffix}
)

# initialize workflow
qlib.init(
    provider_uri=config["qlib_init"]["provider_uri"],
    region=config["qlib_init"]["region"],
)

[1461:MainThread](2021-10-22 22:33:35,478) INFO - qlib.Initialization - [config.py:386] - default_conf: client.
[1461:MainThread](2021-10-22 22:33:35,481) WARNING - qlib.Initialization - [config.py:411] - redis connection failed(host=127.0.0.1 port=6379), DiskExpressionCache and DiskDatasetCache will not be used!
[1461:MainThread](2021-10-22 22:33:35,830) INFO - qlib.Initialization - [__init__.py:56] - qlib successfully initialized based on client settings.
[1461:MainThread](2021-10-22 22:33:35,832) INFO - qlib.Initialization - [__init__.py:58] - data_path={'__DEFAULT_FREQ': PosixPath('/home/us_data_dump')}


In [3]:
latest_dataset = init_instance_by_config(config["task"]["dataset"])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[1461:MainThread](2021-10-22 22:36:05,993) INFO - qlib.timer - [log.py:113] - Time cost: 147.968s | Loading data Done
[1461:MainThread](2021-10-22 22:36:06,012) INFO - qlib.timer - [log.py:113] - Time cost: 0.010s | FilterCol Done
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.8/dist

In [4]:
model = init_instance_by_config(config["task"]["model"])

Please install necessary libs for CatBoostModel.
Please install necessary libs for XGBModel, such as xgboost.


[1461:MainThread](2021-10-22 22:36:06,857) WARNING - qlib.TRA - [pytorch_tra.py:100] - `eval_train` will be ignored when using TRA.router
[1461:MainThread](2021-10-22 22:36:06,862) WARNING - qlib.TRA - [pytorch_tra.py:132] - logdir output/us_158_nassp_epoch100 is not empty
[1461:MainThread](2021-10-22 22:36:06,864) INFO - qlib.TRA - [pytorch_tra.py:141] - init TRAModel...
[1461:MainThread](2021-10-22 22:36:06,873) INFO - qlib.TRA - [pytorch_tra.py:171] - # model params: 59520
[1461:MainThread](2021-10-22 22:36:06,875) INFO - qlib.TRA - [pytorch_tra.py:172] - # tra params: 5606


RNN(
  (rnn): LSTM(20, 64, num_layers=2, batch_first=True)
  (W): Linear(in_features=64, out_features=64, bias=True)
  (u): Linear(in_features=64, out_features=1, bias=False)
)
TRA(
  (predictors): Linear(in_features=128, out_features=3, bias=True)
  (router): LSTM(3, 32, batch_first=True)
  (fc): Linear(in_features=160, out_features=3, bias=True)
)


In [5]:
state_dict = torch.load("/home/qlib_test/examples/benchmarks/TRA/output/us_158_nassp_epoch100/model.bin", map_location="cpu")["model"]

In [6]:
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [7]:
model.fitted = True

In [8]:
pred = model.predict(latest_dataset)

100%|██████████████████████████████████████████████████████████| 9/9 [00:01<00:00,  4.57it/s]
[1461:MainThread](2021-10-22 22:36:08,908) INFO - qlib.TRA - [pytorch_tra.py:516] - test metrics: {'MSE': 0.16148672135989006, 'MAE': 0.3268642527234289, 'IC': 0.03107587425170133, 'ICIR': 0.3446436265046043}


In [9]:
latest_dataset.to_pickle(path="/home/us_dataset_158_100.pkl")

In [10]:
# pred

In [11]:
reset_df = pred.reset_index('instrument')
# reset_df

In [ ]:
from IPython.display import clear_output
import numpy as np
import os
indexs = reset_df.index.drop_duplicates()
cnt = 0 

save_dir = '/home/results_158_100/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

current_buying = {}
top_k = 5
max_buying = 10
num_buy = 0
my_stack = None

data_dir = '/home/us_data_source/'
for idx in indexs:
    # cnt+=1
    # if cnt < 800: continue
    print(idx)
    # today_inform = pd.read_csv(results_dir + str(idx) +'.csv')
    # print(today_inform)
    if type(reset_df.loc[idx]['score']) is np.float32:
        stack = reset_df.loc[idx]
    else:
        stack = reset_df.loc[idx].sort_values(by='score' , ascending=False)
    print(stack)
    
    current_buying_list = list(current_buying.keys())
    for cur_stock in current_buying_list:
        cur_stock_inform = pd.read_csv(data_dir + cur_stock + '.csv')
        today_inform = cur_stock_inform[cur_stock_inform['date'] == str(idx).split(' ')[0]]
        today_stock_price = float(today_inform['open'].round(2))
        current_buying[cur_stock][2] = today_stock_price

    my_stack = stack[stack['instrument'].isin(current_buying_list)]
    my_stack = my_stack.sort_values(by='score' , ascending=False)
    my_stack_list = list(my_stack['instrument'])
    print(my_stack_list)
    print(my_stack)
    
    for x in stack['instrument'][:top_k]:
        print(x)
        if num_buy >= max_buying:
            last_stock = my_stack_list[-1]
            num_last_stock = current_buying[last_stock][0]
            if num_last_stock == 1:
                del current_buying[last_stock]
                my_stack_list = my_stack_list[:-1]
                num_buy -= 1
            else:
                current_buying[last_stock][0] -= 1
                num_buy -= 1
        if x not in current_buying:
            cur_stock = x
            cur_stock_inform = pd.read_csv(data_dir + cur_stock + '.csv')
            today_inform = cur_stock_inform[cur_stock_inform['date'] == str(idx).split(' ')[0]]
            today_stock_price = float(today_inform['open'].round(2))
            current_buying[x] = [1, today_stock_price, today_stock_price] #num, start_price, current_price
            num_buy +=1
        else:
            current_buying[x][0] += 1
            num_buy +=1
        print(num_buy, current_buying)
        input()
        
    print(num_buy, current_buying)
    


    # print(current_buying)
    input()
    clear_output()

2021-10-14 00:00:00
           instrument     score     label   score_0   score_1   score_2
datetime                                                               
2021-10-14          F  0.004292 -0.990971  0.004292  0.035539 -0.088454
2021-10-14         TT  0.002934  1.024563  0.002934  0.075027 -0.048994
2021-10-14       QRVO  0.002173 -0.393029  0.002173  0.038900 -0.002352
2021-10-14        LVS  0.000951 -0.131010 -0.064583  0.030384  0.000951
2021-10-14       TMUS -0.000442 -0.137728 -0.000442  0.045724 -0.022648
...               ...       ...       ...       ...       ...       ...
2021-10-14        EOG -0.156457 -0.399748 -0.034076  0.102862 -0.156457
2021-10-14       COST -0.159157  1.488136 -0.099991  0.026641 -0.159157
2021-10-14      LBTYK -0.164998 -0.231786 -0.100350  0.117584 -0.164998
2021-10-14        PKG -0.173573  0.453495 -0.096019  0.105571 -0.173573
2021-10-14        MRO -0.183396  1.326893 -0.090361  0.081337 -0.183396

[515 rows x 6 columns]
['F', 'QRVO', 'LW', 

TT
10 {'QRVO': [3, 165.15, 162.5], 'BMY': [2, 57.32, 57.07], 'F': [3, 15.11, 15.62], 'LW': [1, 55.8, 56.28], 'TT': [1, 172.05, 172.05]}


QRVO
10 {'QRVO': [4, 165.15, 162.5], 'BMY': [1, 57.32, 57.07], 'F': [3, 15.11, 15.62], 'LW': [1, 55.8, 56.28], 'TT': [1, 172.05, 172.05]}


LVS
10 {'QRVO': [4, 165.15, 162.5], 'F': [3, 15.11, 15.62], 'LW': [1, 55.8, 56.28], 'TT': [1, 172.05, 172.05], 'LVS': [1, 40.57, 40.57]}


TMUS
10 {'QRVO': [4, 165.15, 162.5], 'F': [3, 15.11, 15.62], 'TT': [1, 172.05, 172.05], 'LVS': [1, 40.57, 40.57], 'TMUS': [1, 118.09, 118.09]}


10 {'QRVO': [4, 165.15, 162.5], 'F': [3, 15.11, 15.62], 'TT': [1, 172.05, 172.05], 'LVS': [1, 40.57, 40.57], 'TMUS': [1, 118.09, 118.09]}


In [83]:
current_buying

{'QRVO': [3, 165.15, 159.37],
 'BMY': [1, 57.32, 57.06],
 'ROP': [1, 453.6, 453.16],
 'ATVI': [1, 76.45, 75.34],
 'F': [1, 15.11, 15.59]}

In [45]:
current_buying['QRVO'] = 1

In [37]:
temp_dict

{'ROP': 2, 'ATVI': 2, 'QRVO': 2, 'FCX': 2, 'F': 2}

In [14]:
reset_df.to_pickle(path="/home/reset_df_158_100.pkl")

In [23]:
pred

score     label   score_0   score_1   score_2
datetime   instrument                                                  
2021-10-11 A          -0.059757  1.306738 -0.059757  0.000715 -0.072410
           AAL        -0.098092 -1.689689 -0.098092  0.004654 -0.098579
           AAP        -0.102055 -1.165650 -0.102055  0.066270 -0.122829
           AAPL       -0.087577 -0.796136 -0.087577  0.004984 -0.106730
           ABBV       -0.067606 -0.319126 -0.067606  0.010677 -0.089874
...                         ...       ...       ...       ...       ...
2021-10-21 ZBH        -0.069378       NaN -0.069378  0.003003 -0.130012
           ZBRA       -0.058264       NaN -0.058264  0.016758 -0.029283
           ZION       -0.107694       NaN -0.096385  0.043520 -0.107694
           ZM         -0.013568       NaN  0.013776  0.072926 -0.013568
           ZTS        -0.056932       NaN -0.056932  0.050866 -0.090562

[4635 rows x 5 columns]

In [ ]:
# def backtest(
#     start_time,
#     end_time,
#     strategy,
#     executor,
#     benchmark="SH000300",
#     account=1e9,
#     exchange_kwargs={},
#     pos_type: str = "Position",
# ):